In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_predict

/tmp/ipykernel_1613484/3681729236.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# ORBITAL

In [2]:
crops_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/ORBITAL/mask'
skels = np.load(os.path.join(crops_dir, 'Lskeleton.npy'))
skel_subs = pd.read_csv(os.path.join(crops_dir, 'Lskeleton_subject.csv'))
skels = skels.reshape(skels.shape[0], np.prod(skels.shape[1:]))
skels = skels.astype(bool)

In [3]:
splits_dir = '/neurospin/dico/data/deep_folding/current/datasets/orbital_patterns/Troiani'
train_subs = pd.read_csv(os.path.join(splits_dir, 'split_0-6.csv'), names=['Subject'])
val_subs = pd.read_csv(os.path.join(splits_dir, 'split_7.csv'), names=['Subject'])
test_subs = pd.read_csv(os.path.join(splits_dir, 'split_8-9.csv'), names=['Subject'])

In [4]:
label='Left_OFC'
labels = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_OFC_labels_from_0.csv', usecols=['Subject', label])
proportions = np.unique(labels[label], return_counts=True)
proportions = proportions[1] / np.sum(proportions[1])

In [5]:
train = skel_subs.loc[skel_subs['Subject'].isin(train_subs['Subject'])]
idxs_train = train.index.tolist()
Y_train = pd.merge(train, labels)[label]
train_skels= skels[idxs_train]

val = skel_subs.loc[skel_subs['Subject'].isin(val_subs['Subject'])]
idxs_val = val.index.tolist()
Y_val = pd.merge(val, labels)[label]
val_skels= skels[idxs_val]

test = skel_subs.loc[skel_subs['Subject'].isin(test_subs['Subject'])]
idxs_test = test.index.tolist()
Y_test = pd.merge(test, labels)[label]
test_skels= skels[idxs_test]

In [9]:
np.var(train_skels) * train_skels.shape[1]

650.9700082294804

In [13]:
#for C in [1e-2, 1e-1, 1, 10]: 
for C in [1]:   
    for class_weight in [None]:
        for degree in [3]:
            for gamma in [k/(np.var(train_skels) * train_skels.shape[1]) for k in [0.1, 0.33, 3.3, 10]]:
                model = SVC(kernel='rbf', probability=True, degree=degree, gamma=gamma,
                            max_iter=10000, random_state=None,
                            C=C, class_weight=class_weight, decision_function_shape='ovr')
                
                model.fit(train_skels, Y_train)
                labels_proba = model.predict_proba(val_skels)
                roc_aucs = roc_auc_score(Y_val, labels_proba, multi_class='ovr', average=None)

                print(gamma, C, class_weight)
                print(roc_aucs)
                print(np.dot(roc_aucs, proportions.T))

0.00015361690820746374 1 None
[0.60125    0.53496503 0.72222222 0.39150943]
0.5910736128709299
0.0005069357970846303 1 None
[0.59875    0.47377622 0.72698413 0.39622642]
0.574122560397249
0.005069357970846303 1 None
[0.54125    0.6048951  0.26984127 0.58490566]
0.5140037152614815
0.015361690820746373 1 None
[0.39       0.44055944 0.36349206 0.46698113]
0.40409526900583437


In [127]:
# test
model = SVC(kernel='rbf', probability=True, degree=3,
                        max_iter=-1, random_state=None,
                        C=1, class_weight=None, decision_function_shape='ovr')
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(test_skels)
roc_auc = roc_auc_score(Y_test, labels_proba, multi_class='ovr', average=None)
print(roc_auc)
print(np.dot(roc_auc, proportions.T))

[0.66545675 0.61278195 0.65291262 0.70560748]
0.6512508148626943


In [128]:
# LogisticRegression
model = LogisticRegression(C=1,  penalty='l1', solver='liblinear', class_weight='balanced', max_iter=100, multi_class='ovr')
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(val_skels)
roc_auc = roc_auc_score(Y_val, labels_proba, multi_class='ovr', average=None)
print(roc_auc)
print(np.dot(roc_auc, proportions.T))

[0.62       0.51573427 0.63968254 0.36792453]
0.5789860793389128


In [124]:
#test 
model = LogisticRegression(C=1,  penalty='l1', solver='liblinear', class_weight='balanced', max_iter=100, multi_class='ovr')
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(test_skels)
roc_auc = roc_auc_score(Y_test, labels_proba, multi_class='ovr', average=None)
print(roc_auc)
print(np.dot(roc_auc, proportions.T))

[0.65759226 0.60594668 0.59061489 0.42523364]
0.6171394484409242


In [129]:
# ElasticNet
model = LogisticRegression(C=1, penalty='elasticnet', solver='saga', l1_ratio=0.5, class_weight='balanced', max_iter=100, multi_class='ovr')
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(val_skels)
roc_auc = roc_auc_score(Y_val, labels_proba, multi_class='ovr', average=None)
print(roc_auc)
print(np.dot(roc_auc, proportions.T))

/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[0.63       0.56643357 0.73015873 0.36320755]
0.6133699142237123


In [126]:
# ElasticNet
model = LogisticRegression(C=0.1, penalty='elasticnet', solver='saga', l1_ratio=0.5, class_weight='balanced', max_iter=100, multi_class='ovr')
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(test_skels)
roc_auc = roc_auc_score(Y_test, labels_proba, multi_class='ovr', average=None)
print(roc_auc)
print(np.dot(roc_auc, proportions.T))

/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[0.6291591  0.58202324 0.64967638 0.47079439]
0.6098809629161359


/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


# PCS

In [11]:
crops_dir = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/crops/2mm/CINGULATE/mask'
skels = np.load(os.path.join(crops_dir, 'Rskeleton.npy'))
skel_subs = pd.read_csv(os.path.join(crops_dir, 'Rskeleton_subject.csv'))
skels = skels.reshape(skels.shape[0], np.prod(skels.shape[1:]))
skels = skels.astype(bool)

In [12]:
splits_dir = '/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/train_val_test'
train_subs = pd.read_csv(os.path.join(splits_dir, 'ACCpatterns_subjects_filtered_train.csv'), names=['Subject'])
val_subs = pd.read_csv(os.path.join(splits_dir, 'ACCpatterns_subjects_filtered_val.csv'), names=['Subject'])
test_subs = pd.read_csv(os.path.join(splits_dir, 'ACCpatterns_subjects_filtered_test.csv'), names=['Subject'])

In [13]:
label='Right_PCS'
labels = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/subjects_labels.csv', usecols=['long_name', label])
labels.columns = ['Subject', label]
proportions = np.unique(labels[label], return_counts=True)
proportions = proportions[1] / np.sum(proportions[1])

In [14]:
train = skel_subs.loc[skel_subs['Subject'].isin(train_subs['Subject'])]
idxs_train = train.index.tolist()
Y_train = pd.merge(train, labels)[label]
train_skels= skels[idxs_train]

val = skel_subs.loc[skel_subs['Subject'].isin(val_subs['Subject'])]
idxs_val = val.index.tolist()
Y_val = pd.merge(val, labels)[label]
val_skels= skels[idxs_val]

test = skel_subs.loc[skel_subs['Subject'].isin(test_subs['Subject'])]
idxs_test = test.index.tolist()
Y_test = pd.merge(test, labels)[label]
test_skels= skels[idxs_test]

In [41]:
for C in [1e-2, 1e-1, 1, 10]:    
    for class_weight in ['balanced']:
        for degree in [2, 3, 4, 5]:

            model = SVC(kernel='rbf', probability=True, degree=degree,
                        max_iter=-1, random_state=None,
                        C=C, class_weight=class_weight, decision_function_shape='ovr')
            
            model.fit(train_skels, Y_train)
            labels_proba = model.predict_proba(val_skels)
            roc_auc = roc_auc_score(Y_val, labels_proba[:, 1])

            print(C, class_weight, degree)
            print(roc_auc)

0.01 balanced 2
0.1904761904761905
0.01 balanced 3
0.1904761904761905
0.01 balanced 4
0.1904761904761905
0.01 balanced 5
0.1904761904761905
0.1 balanced 2
0.1904761904761905
0.1 balanced 3
0.1904761904761905
0.1 balanced 4
0.1904761904761905
0.1 balanced 5
0.1904761904761905
1 balanced 2
0.8205128205128205
1 balanced 3
0.8205128205128205
1 balanced 4
0.8205128205128205
1 balanced 5
0.8205128205128205
10 balanced 2
0.8241758241758242
10 balanced 3
0.8241758241758242
10 balanced 4
0.8241758241758242
10 balanced 5
0.8241758241758242


In [ ]:
model = SVC(kernel='rbf', probability=True, degree=3,
                        max_iter=100, random_state=None,
                        C=1, class_weight='balanced', decision_function_shape='ovr')
model.fit(val_skels, Y_val)
labels_proba = model.predict_proba(val_skels)
roc_auc = roc_auc_score(Y_val, labels_proba[:, 1])
print(roc_auc)

In [39]:
model = SVC(kernel='rbf', probability=True, degree=3,
                        max_iter=-1, random_state=None,
                        C=1, class_weight='balanced', decision_function_shape='ovr')
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(test_skels)
roc_auc = roc_auc_score(Y_test, labels_proba[:, 1])
print(roc_auc)

/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


0.7428842504743833


In [71]:
# LogisticRegression
model = LogisticRegression(C=0.3, penalty='l1', solver='liblinear', class_weight='balanced', max_iter=100000)
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(val_skels)
roc_auc = roc_auc_score(Y_val, labels_proba[:, 1])
print(roc_auc)

0.7912087912087913


In [76]:
#test
model = LogisticRegression(C=0.3, penalty='l1', solver='liblinear', class_weight='balanced', max_iter=100000)
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(test_skels)
roc_auc = roc_auc_score(Y_test, labels_proba[:, 1])
print(roc_auc)

0.6195445920303605


In [58]:
# ElasticNet
model = LogisticRegression(C=0.1, penalty='elasticnet', solver='saga', l1_ratio=0.5, class_weight='balanced', max_iter=100)
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(val_skels)
roc_auc = roc_auc_score(Y_val, labels_proba[:, 1])
print(roc_auc)

0.8058608058608059


/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [77]:
# test
model = LogisticRegression(C=0.1, penalty='elasticnet', solver='saga', l1_ratio=0.5, class_weight='balanced', max_iter=100)
model.fit(train_skels, Y_train)
labels_proba = model.predict_proba(test_skels)
roc_auc = roc_auc_score(Y_test, labels_proba[:, 1])
print(roc_auc)

0.7201138519924098


/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
